## Extracting Data from Socialverse API and Saving to CSV Files


In [ ]:
import requests
import pandas as pd
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
flic = user_secrets.get_secret("FLICK_TOKEN")


response = requests.get("https://api.socialverseapp.com/posts/view?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")
data_1 = response.json()['posts']
df_1 = pd.DataFrame(data_1)
df_1.to_csv('viewed.csv',index = False)
response = requests.get("https://api.socialverseapp.com/posts/like?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")
data_2 = response.json()['posts']
df_2 = pd.DataFrame(data_2)
df_2.to_csv('Liked.csv',index=False)
response = requests.get("https://api.socialverseapp.com/posts/inspire?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")
data_3 = response.json()['posts']
df_3 = pd.DataFrame(data_3)
df_3.to_csv('Inspired.csv',index=False)
response = requests.get("https://api.socialverseapp.com/posts/rating?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if")
data_4 = response.json()['posts']
df_4 = pd.DataFrame(data_4)
df_4.to_csv('Rated.csv')

url = "https://api.socialverseapp.com/posts/summary/get?page=1&page_size=1000"

headers = {
    "Flic-Token": flic
}
response = requests.get(url=url,headers = headers)
data_5 = response.json()['posts']
df_5 = pd.DataFrame(data_5)
df_5.to_csv('Posts.csv',index = False)

response = requests.get("https://api.socialverseapp.com/users/get_all?page=1&page_size=1000",headers = headers)
data_6 = response.json()['posts']
df_6 = pd.DataFrame(data_6)
df_6.to_csv('User_Demographics.csv',index = False)

# Loading the written datasets from the previous code cell 

In [83]:
import pandas as pd
import requests

User_demographics = pd.read_csv("/kaggle/input/my-new-datastets/Demographics.csv")
Inspired = pd.read_csv("/kaggle/input/my-new-datastets/Inspired.csv")
Liked = pd.read_csv("/kaggle/input/my-new-datastets/Liked.csv")
Rated= pd.read_csv("/kaggle/input/my-new-datastets/Rated.csv")
viewed = pd.read_csv("/kaggle/input/my-data/viewed.csv")
posts = pd.read_csv("/kaggle/input/another-dataset/Posts.csv")

# Inspecting the columns of each datasets

In [84]:
print(User_demographics.columns)
print(Inspired.columns)
print(Liked.columns)
print(Rated.columns)
print(viewed.columns)
print(posts.columns)

print(User_demographics.info())
print(Inspired.info())
print(Liked.info())
print(Rated.info())
print(viewed.info())
print(posts.info())

Index(['id', 'first_name', 'last_name', 'username', 'email', 'role',
       'profile_url', 'bio', 'website_url', 'instagram-url', 'youtube_url',
       'tictok_url', 'isVerified', 'referral_code', 'has_wallet', 'last_login',
       'share_count', 'post_count', 'following_count', 'follower_count',
       'is_verified', 'is_online', 'latitude', 'longitude'],
      dtype='object')
Index(['id', 'post_id', 'user_id', 'inspired_at'], dtype='object')
Index(['id', 'post_id', 'user_id', 'liked_at'], dtype='object')
Index(['id', 'post_id', 'user_id', 'rating_percent', 'rated_at'], dtype='object')
Index(['id', 'post_id', 'user_id', 'viewed_at'], dtype='object')
Index(['id', 'category', 'slug', 'title', 'identifier', 'comment_count',
       'upvote_count', 'view_count', 'exit_count', 'rating_count',
       'average_rating', 'share_count', 'video_link', 'contract_address',
       'chain_id', 'chart_url', 'baseToken', 'is_locked', 'created_at',
       'first_name', 'last_name', 'username', 'upvoted'

# Implementing a feedback loop and performing necessary preprocessing strategies , handling inconsistent datatypes 

In [85]:
import pandas as pd
import numpy as np
import re


def load_dataset(file_path):
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        print(f"Error loading dataset: {str(e)}")
        return None


def identify_missing_columns(df):
    missing_columns = df.columns[df.isnull().sum() > 0.5*len(df)].tolist()
    return missing_columns

def fill_missing_values(df, column, value):
    if column in df.columns:
        df[column] = df[column].fillna(value)
    return df

def drop_missing_columns(df, columns):
    if isinstance(columns, list):
        df = df.drop(columns=columns, axis=1)
    return df

def validate_data_formats(df):
    def validate_email(email):
        pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
        return bool(re.match(pattern, email))

    def validate_url(url):
        pattern = r"^https?:\/\/(?:[-\w]+\.)+[a-zA-Z]{2,}(?:\/[^\s]*)?$"
        return bool(re.match(pattern, url))

    def validate_coordinates(lat, lon):
        try:
            float(lat)
            float(lon)
            return float(lat) >= -90 and float(lat) <= 90 and float(lon) >= -180 and float(lon) <= 180
        except ValueError:
            return False

    for index, row in df.iterrows():
        if 'email' in row and not validate_email(row['email']):
            print(f"Email format is invalid for record {index}: {row['email']}")
        if 'profile_url' in row and not validate_url(row['profile_url']):
            print(f"Profile URL format is invalid for record {index}: {row['profile_url']}")
        if 'website_url' in row and not validate_url(row['website_url']):
            print(f"Website URL format is invalid for record {index}: {row['website_url']}")
        if 'instagram-url' in row and not validate_url(row['instagram-url']):
            print(f"Instagram URL format is invalid for record {index}: {row['instagram-url']}")
        if 'youtube_url' in row and not validate_url(row['youtube_url']):
            print(f"YouTube URL format is invalid for record {index}: {row['youtube_url']}")
        if 'tictok_url' in row and not validate_url(row['tictok_url']):
            print(f"Tictok URL format is invalid for record {index}: {row['tictok_url']}")
        if 'latitude' in row and 'longitude' in row and not validate_coordinates(row['latitude'], row['longitude']):
            print(f"Latitude or longitude format is invalid for record {index}: {row['latitude']}, {row['longitude']}")

missing_columns = identify_missing_columns(User_demographics)

print(f"Columns with more than 50% missing values: {missing_columns}")


action = input("Do you want to fill the missing values or drop the columns? (fill/drop): ")

if action.lower() == 'fill':
    for column in missing_columns:
        User_demographics = fill_missing_values(User_demographics, column, value)
    print("Missing values filled successfully.")
elif action.lower() == 'drop':
    confirm = input(f"Are you sure you want to drop columns {missing_columns}? (yes/no): ")
    if confirm.lower() == 'yes':
        User_demographics = drop_missing_columns(User_demographics, missing_columns)
        print("Columns dropped successfully.")
    else:
        print("Operation cancelled.")
else:
    print("Invalid action selected.")

validate_data_formats(User_demographics)

Columns with more than 50% missing values: ['bio', 'website_url', 'instagram-url', 'youtube_url', 'tictok_url', 'referral_code', 'latitude', 'longitude']


Do you want to fill the missing values or drop the columns? (fill/drop):  drop
Are you sure you want to drop columns ['bio', 'website_url', 'instagram-url', 'youtube_url', 'tictok_url', 'referral_code', 'latitude', 'longitude']? (yes/no):  yes


Columns dropped successfully.
Email format is invalid for record 3: code#gmail
Email format is invalid for record 6: codewitha#gmail


# Manually Correcting other errors

In [86]:
User_demographics.loc[3,'email'] = 'code@gmail.com'
User_demographics.loc[6,'email'] = 'codewith@gmail.com'

In [87]:
User_demographics.last_login = User_demographics['last_login'].fillna(User_demographics.last_login.mode()[0])
User_demographics.isna().sum()

id                 0
first_name         0
last_name          0
username           0
email              0
role               0
profile_url        0
isVerified         0
has_wallet         0
last_login         0
share_count        0
post_count         0
following_count    0
follower_count     0
is_verified        0
is_online          0
dtype: int64

# Converting boolean values to Integer

In [88]:
bools = [col for col in User_demographics.select_dtypes(include = 'bool')]
User_demographics[bools] = User_demographics[bools].astype(float)
User_demographics

,id,first_name,last_name,username,email,role,profile_url,isVerified,has_wallet,last_login,share_count,post_count,following_count,follower_count,is_verified,is_online
0,1,Michael,Dadzie,afrobeezy,dadziemikke@outlook.com,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,0,57,8,20,0.0,1.0
1,2,John,Doe,doey,test@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
2,3,John,Doe,doeyy,test1@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
3,4,John,Doe,doeyyy,code@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
4,5,Sachin,Kinha,kinha,sachinkinha1913@gmail.com,A,https://assets.socialverseapp.com/profile/kinh...,0.0,1.0,2024-12-09 04:51:50,0,947,17,47,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1015,yjnywzpbwc,privaterelay.appleid.com,yjnywzpbwc_08c9i6l7f,yjnywzpbwc@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/15.png,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
996,1016,xkzvdj57xj,privaterelay.appleid.com,xkzvdj57xj_09f7c9l8i,xkzvdj57xj@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/16.png,0.0,0.0,2024-10-08 11:21:08,0,0,0,0,0.0,0.0
997,1017,Nuage,Laboratoire,nuage_08i0l4c5f,xkqrhcbo7am2slbnr23lfpyzy4-00@cloudtestlabacco...,U,https://assets.socialverseapp.com/profile/16.png,0.0,0.0,2024-11-20 12:32:38,0,0,0,0,0.0,0.0
998,1018,2mhdbqqk6q,privaterelay.appleid.com,2mhdbqqk6q_06l6f3i1c,2mhdbqqk6q@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/1.png,0.0,0.0,2024-10-09 11:13:44,0,0,0,1,0.0,0.0


# Verifying the presence of null values in the post dataset : 

In [91]:
posts.isna().sum()

id                     0
category               0
slug                   0
title                322
identifier             0
comment_count          0
upvote_count           0
view_count             0
exit_count             0
rating_count           0
average_rating         0
share_count            0
video_link             0
contract_address     932
chain_id             932
chart_url            932
baseToken              0
is_locked              0
created_at             0
first_name             0
last_name              0
username               0
upvoted                0
bookmarked             0
thumbnail_url          0
gif_thumbnail_url    883
following              0
picture_url            0
post_summary           0
dtype: int64

# Inspecting the unique records present in the datasets

In [73]:
User_demographics.nunique()

id                 1000
first_name          874
last_name           468
username           1000
email               999
role                  2
profile_url          60
isVerified            1
has_wallet            2
last_login          747
share_count           1
post_count            9
following_count      14
follower_count       13
is_verified           1
is_online             2
dtype: int64

# Inconsistent slug names of post titles

In [92]:
posts['slug']

0                       recipe-for-a-flow-state
1                                    why-fit-in
2                      transcending-singularity
3                              peak-performance
4                     our-existential-situation
                         ...                   
995    ece97b1efa78b1042eaa21170d38f83a006103b6
996    ce76de5e51d2c4590c9f1fd947c1f2dffcb83938
997    bdc13af36b85c9170d6b7bd29e48009719220441
998    5bcf09ef8e1557548f2b3ca645f034f571d3e420
999    b2b57eaad22b99cab4b53d1c3405cf75d10d884e
Name: slug, Length: 1000, dtype: object

# adding a delimiter '-' and preprocessing the slug attribute of the posts dataset

In [93]:
def generate_slug(title):
    if isinstance(title, str):  # Check if title is a string
        return '-'.join(title.lower().strip().split())
    else:
        return ''  # Return an empty string or some other placeholder for non-strings

posts['slug'] = posts.apply(
    lambda row: generate_slug(row['title']) if row['slug'] != generate_slug(row['title']) else row['slug'],
    axis=1
)

In [76]:
posts['slug'][990:999]

990                                                  💯💯💯
991    peak-beauty-of-nature.....................#sun...
992                                              snow!!!
993    dancing-cat---@visiontoinspire-#dodge-#hellcat...
994    the-perfect-conditions-in-norway-.would-you-ta...
995    kalsubai-is-the-highest-peak-in-maharashtra,-s...
996    tap-into-the-magic-of-winter.-.sometimes,-all-...
997       anzeige----️from-idea-to-masterpiece-️with-the
998    in-the-heart-of-darkness,-magic-is-our-only-sh...
Name: slug, dtype: object

# Parsing the category column 

In [94]:
import ast
import pandas as pd

def safe_parse_category(x):
    try:
        x = x.replace('\\\'', '\'').replace('\\n', '')
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return None

posts['category'] = posts['category'].apply(safe_parse_category)

posts['category_name'] = posts['category'].apply(lambda x: x['name'] if x else None)


posts['category_name'][1:5]

1    Vible
2    Vible
3    Vible
4    Vible
Name: category_name, dtype: object

In [78]:
posts['category_name']

0      Vible
1      Vible
2      Vible
3      Vible
4      Vible
       ...  
995     Flic
996     Flic
997     Flic
998     Flic
999     Flic
Name: category_name, Length: 1000, dtype: object

In [95]:
posts = posts.drop(columns = ['category'],axis = 1)

In [96]:
posts = posts.drop(columns = ['contract_address','chain_id','chart_url','gif_thumbnail_url'],axis = 1)
posts.isna().sum()

id                  0
slug                0
title             322
identifier          0
comment_count       0
upvote_count        0
view_count          0
exit_count          0
rating_count        0
average_rating      0
share_count         0
video_link          0
baseToken           0
is_locked           0
created_at          0
first_name          0
last_name           0
username            0
upvoted             0
bookmarked          0
thumbnail_url       0
following           0
picture_url         0
post_summary        0
category_name       0
dtype: int64

In [97]:
posts['title'] = posts['title'].fillna(posts['title'].mode()[0])
posts.isna().sum()

id                0
slug              0
title             0
identifier        0
comment_count     0
upvote_count      0
view_count        0
exit_count        0
rating_count      0
average_rating    0
share_count       0
video_link        0
baseToken         0
is_locked         0
created_at        0
first_name        0
last_name         0
username          0
upvoted           0
bookmarked        0
thumbnail_url     0
following         0
picture_url       0
post_summary      0
category_name     0
dtype: int64

In [98]:
posts['post_summary'].values[:5]

array(['{\'actions\': {\'key_events\': [\'discussion about flow state\', \'demonstration of body language and gestures\', \'showcasing examples of outdoor activities\']}, \'audio_elements\': {\'descriptions\': [\'synchronized audio and visual elements\', \'animated speaking style\', \'emotional peaks corresponding to content\']}, \'description\': \'The video unfolds as a dynamic exploration of the concept of "flow state," complementing the audio transcription with visual storytelling. The main speaker, a male individual, captures the viewer’s attention through expressive facial gestures and animated speech, embodying the theme of immersion and presence. His body language is energetic and open, illustrating his passion for the subject matter.\\n\\nAs the speaker shares insights on achieving a flow state, the visuals alternate between close-up shots of his face and mid-range views that allow for hand gestures, such as pointing and emphasizing key points. His expressions change from conte

In [99]:
import pandas as pd
import ast

posts['post_summary'] = posts['post_summary'].apply(ast.literal_eval)

posts['description'] = posts['post_summary'].apply(lambda x: x.get('description', 'No description available'))
posts['actions'] = posts['post_summary'].apply(lambda x: x.get('actions', 'No actions available'))
posts['emotions'] = posts['post_summary'].apply(lambda x: x.get('emotions', 'No emotions available'))


posts[['description', 'actions', 'emotions']]

,description,actions,emotions
0,The video unfolds as a dynamic exploration of ...,"{'key_events': ['discussion about flow state',...","{'conveyed': ['enthusiasm', 'engagement', 'cur..."
1,The video is structured as an emotional and in...,{'key_actions': ['Character expressing inner t...,"{'expressed_emotions': ['Anger', 'Determinatio..."
2,The video presents a compelling visual narrati...,{'key_actions': ['Speaker engaging in animated...,"{'emotional_content': ['Wonder', 'Intensity', ..."
3,The video presents a seamless narrative explor...,"{'key_actions': ['speaker uses hand gestures',...","{'moods': ['excitement', 'passion', 'contempla..."
4,"The video opens with a male speaker, who has a...",{'key_actions': ['Speaker engages with the cam...,"{'conveyed_emotions': ['Concern', 'Passion', '..."
...,...,...,...
995,I'm unable to analyze or interpret video frame...,{'main_action': 'suggesting the creation of st...,"{'conveyed_emotions': ['encouraging', 'support..."
996,"As the video unfolds, it presents a cohesive n...",{'key_actions': ['walking through a vast open ...,"{'range_of_emotions': ['deep contemplation', '..."
997,"In the video, a cohesive narrative unfolds thr...","{'key_actions': ['Adjusting headphones', 'Sket...",{'emotional_expressions': ['Focused determinat...
998,The video unfolds as a cinematic narrative set...,{'initial_actions': ['Carrying an old-fashione...,"{'expressed': ['anticipation', 'concern', 'exc..."


In [100]:
posts = posts.drop(columns = 'post_summary',axis = 1)

In [101]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1000 non-null   int64 
 1   slug            1000 non-null   object
 2   title           1000 non-null   object
 3   identifier      1000 non-null   object
 4   comment_count   1000 non-null   int64 
 5   upvote_count    1000 non-null   int64 
 6   view_count      1000 non-null   int64 
 7   exit_count      1000 non-null   int64 
 8   rating_count    1000 non-null   int64 
 9   average_rating  1000 non-null   int64 
 10  share_count     1000 non-null   int64 
 11  video_link      1000 non-null   object
 12  baseToken       1000 non-null   object
 13  is_locked       1000 non-null   bool  
 14  created_at      1000 non-null   int64 
 15  first_name      1000 non-null   object
 16  last_name       1000 non-null   object
 17  username        1000 non-null   object
 18  upvoted  

In [102]:
viewed['post_id'].unique

<bound method Series.unique of 0      631
1      626
2      571
3      558
4      551
      ... 
995    161
996    143
997    151
998    159
999    560
Name: post_id, Length: 1000, dtype: int64>

In [31]:
Liked['liked_at'] = pd.to_datetime(Liked['liked_at'])
Liked

,id,post_id,user_id,liked_at
0,2,26,9,2023-10-24 19:28:41
1,3,33,9,2023-10-26 09:08:32
2,5,36,1,2023-10-28 09:08:35
3,7,36,16,2023-10-28 09:15:57
4,9,52,9,2023-10-30 15:48:46
...,...,...,...,...
995,1623,1062,912,2024-08-09 13:15:14
996,1625,1104,917,2024-08-09 13:27:55
997,1626,1098,917,2024-08-09 13:28:08
998,1627,1099,1,2024-08-09 16:44:25


In [32]:
Rated['rated_at'] = pd.to_datetime(Rated['rated_at'])
Rated

,id,post_id,user_id,rating_percent,rated_at
0,1,516,1,64,2024-01-11 12:45:41
1,2,160,1,62,2024-01-11 12:47:15
2,3,19,1,20,2024-01-11 12:47:28
3,4,148,1,100,2024-01-11 12:49:35
4,5,513,1,63,2024-01-11 12:50:58
...,...,...,...,...,...
995,1036,142,487,20,2024-06-20 09:58:48
996,1037,516,487,19,2024-06-20 09:58:49
997,1038,150,487,19,2024-06-20 09:58:51
998,1039,779,487,17,2024-06-20 09:58:52


In [33]:
User_demographics

,id,first_name,last_name,username,email,role,profile_url,isVerified,has_wallet,last_login,share_count,post_count,following_count,follower_count,is_verified,is_online
0,1,Michael,Dadzie,afrobeezy,dadziemikke@outlook.com,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,0,57,8,20,0.0,1.0
1,2,John,Doe,doey,test@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
2,3,John,Doe,doeyy,test1@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
3,4,John,Doe,doeyyy,code@gmail.com,U,https://cdn-assets.socialverseapp.com/profile/...,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
4,5,Sachin,Kinha,kinha,sachinkinha1913@gmail.com,A,https://assets.socialverseapp.com/profile/kinh...,0.0,1.0,2024-12-09 04:51:50,0,947,17,47,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1015,yjnywzpbwc,privaterelay.appleid.com,yjnywzpbwc_08c9i6l7f,yjnywzpbwc@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/15.png,0.0,0.0,2024-01-10 07:21:31,0,0,0,0,0.0,0.0
996,1016,xkzvdj57xj,privaterelay.appleid.com,xkzvdj57xj_09f7c9l8i,xkzvdj57xj@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/16.png,0.0,0.0,2024-10-08 11:21:08,0,0,0,0,0.0,0.0
997,1017,Nuage,Laboratoire,nuage_08i0l4c5f,xkqrhcbo7am2slbnr23lfpyzy4-00@cloudtestlabacco...,U,https://assets.socialverseapp.com/profile/16.png,0.0,0.0,2024-11-20 12:32:38,0,0,0,0,0.0,0.0
998,1018,2mhdbqqk6q,privaterelay.appleid.com,2mhdbqqk6q_06l6f3i1c,2mhdbqqk6q@privaterelay.appleid.com,U,https://assets.socialverseapp.com/profile/1.png,0.0,0.0,2024-10-09 11:13:44,0,0,0,1,0.0,0.0


In [43]:
bools = [col for col in posts.select_dtypes(include = 'bool')]
posts[bools] = posts[bools].astype(float)

In [103]:
print(posts.info())
print(User_demographics.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1000 non-null   int64 
 1   slug            1000 non-null   object
 2   title           1000 non-null   object
 3   identifier      1000 non-null   object
 4   comment_count   1000 non-null   int64 
 5   upvote_count    1000 non-null   int64 
 6   view_count      1000 non-null   int64 
 7   exit_count      1000 non-null   int64 
 8   rating_count    1000 non-null   int64 
 9   average_rating  1000 non-null   int64 
 10  share_count     1000 non-null   int64 
 11  video_link      1000 non-null   object
 12  baseToken       1000 non-null   object
 13  is_locked       1000 non-null   bool  
 14  created_at      1000 non-null   int64 
 15  first_name      1000 non-null   object
 16  last_name       1000 non-null   object
 17  username        1000 non-null   object
 18  upvoted  

# Merging user demographics and posts with username as common column

In [104]:
combined_df = pd.merge(posts, User_demographics, on='username', how='inner')
combined_df.isna().sum()

id_x               0
slug               0
title              0
identifier         0
comment_count      0
upvote_count       0
view_count         0
exit_count         0
rating_count       0
average_rating     0
share_count_x      0
video_link         0
baseToken          0
is_locked          0
created_at         0
first_name_x       0
last_name_x        0
username           0
upvoted            0
bookmarked         0
thumbnail_url      0
following          0
picture_url        0
category_name      0
description        0
actions            1
emotions           1
id_y               0
first_name_y       0
last_name_y        0
email              0
role               0
profile_url        0
isVerified         0
has_wallet         0
last_login         0
share_count_y      0
post_count         0
following_count    0
follower_count     0
is_verified        0
is_online          0
dtype: int64

In [105]:
combined_df['username'].unique()

array(['afrobeezy', 'mike', 'jack', 'kinha', 'swapnil', 'tarak', 'mridul',
       'ybctjr2q9r_01l6i9f7c', 'daksh_bhardwaj',
       'mohamedmostafffa_07i8f5l1c', 'michaeldadziie', 'shivam'],
      dtype=object)

# Dropping the repitive columns that were formed during merge

In [106]:
combined_df.drop(columns=[col for col in combined_df.columns if col.endswith('_y')], inplace=True)
combined_df.rename(columns={'share_count_x': 'share_count', 'first_name_x': 'first_name', 'last_name_x': 'last_name'}, inplace=True)
combined_df.fillna({'role': 'Unknown'}, inplace=True)
combined_df

,id_x,slug,title,identifier,comment_count,upvote_count,view_count,exit_count,rating_count,average_rating,...,role,profile_url,isVerified,has_wallet,last_login,post_count,following_count,follower_count,is_verified,is_online
0,11,recipe-for-a-flow-state,Recipe for a flow state,OSsJAMz,0,45,63,0,7,36,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
1,12,why-fit-in..?,Why fit in..?,iGqBOIW,0,42,54,0,0,0,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
2,13,transcending-singularity,Transcending Singularity,QAUE7s4,0,20,75,0,45,8,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
3,14,peak-performance?,Peak Performance?,7bKiXIe,0,71,95,6,6,28,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
4,15,our-existential-situation,Our Existential Situation,BLFD9FX,0,64,72,0,9,33,...,A,https://assets.socialverseapp.com/profile/afro...,0.0,1.0,2024-12-07 12:42:06,57,8,20,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1332,"kalsubai-is-the-highest-peak-in-maharashtra,-s...","Kalsubai is the highest peak in Maharashtra, s...",5A0oaCu,0,23,54,49,2,31,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0
996,1334,"tap-into-the-magic-of-winter.-.sometimes,-all-...","Tap into the magic of winter. .Sometimes, all ...",15NGhqi,0,7,88,53,2,23,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0
997,1335,anzeige----️from-idea-to-masterpiece-️with-the,anzeige -- ️From idea to masterpiece ️with the,ICwNbra,0,24,90,88,2,16,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0
998,1336,"in-the-heart-of-darkness,-magic-is-our-only-sh...","In the heart of darkness, magic is our only sh...",IQ-Mqig,0,5,83,42,2,12,...,U,https://assets.socialverseapp.com/profile/shiv...,0.0,1.0,2024-12-06 06:32:01,19,8,8,0.0,0.0


In [107]:
combined_df = combined_df.dropna()
combined_df.isna().sum()

id_x               0
slug               0
title              0
identifier         0
comment_count      0
upvote_count       0
view_count         0
exit_count         0
rating_count       0
average_rating     0
share_count        0
video_link         0
baseToken          0
is_locked          0
created_at         0
first_name         0
last_name          0
username           0
upvoted            0
bookmarked         0
thumbnail_url      0
following          0
picture_url        0
category_name      0
description        0
actions            0
emotions           0
email              0
role               0
profile_url        0
isVerified         0
has_wallet         0
last_login         0
post_count         0
following_count    0
follower_count     0
is_verified        0
is_online          0
dtype: int64

# Dropping more columns

In [108]:
columns_to_drop = [
    'id_x', 'slug', 'identifier', 'video_link', 'baseToken', 'is_locked', 'created_at', 
    'first_name', 'last_name', 'is_verified', 'is_online', 'profile_url', 'last_login','picture_url', 'actions', 'email', 
    'role'
]

filtered_df = combined_df.drop(columns=columns_to_drop)

print(filtered_df.head())

                       title  comment_count  upvote_count  view_count  \
0    Recipe for a flow state              0            45          63   
1              Why fit in..?              0            42          54   
2   Transcending Singularity              0            20          75   
3          Peak Performance?              0            71          95   
4  Our Existential Situation              0            64          72   

   exit_count  rating_count  average_rating  share_count   username  upvoted  \
0           0             7              36            8  afrobeezy     True   
1           0             0               0            0  afrobeezy    False   
2           0            45               8            0  afrobeezy    False   
3           6             6              28            0  afrobeezy    False   
4           0             9              33            0  afrobeezy    False   

   ...                                      thumbnail_url following  \
0  ...  h

In [109]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999 entries, 0 to 999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            999 non-null    object 
 1   comment_count    999 non-null    int64  
 2   upvote_count     999 non-null    int64  
 3   view_count       999 non-null    int64  
 4   exit_count       999 non-null    int64  
 5   rating_count     999 non-null    int64  
 6   average_rating   999 non-null    int64  
 7   share_count      999 non-null    int64  
 8   username         999 non-null    object 
 9   upvoted          999 non-null    bool   
 10  bookmarked       999 non-null    bool   
 11  thumbnail_url    999 non-null    object 
 12  following        999 non-null    bool   
 13  category_name    999 non-null    object 
 14  description      999 non-null    object 
 15  emotions         999 non-null    object 
 16  isVerified       999 non-null    float64
 17  has_wallet       999 

# How are emotions stored

In [110]:
filtered_df['emotions'][1:5]

1    {'expressed_emotions': ['Anger', 'Determinatio...
2    {'emotional_content': ['Wonder', 'Intensity', ...
3    {'moods': ['excitement', 'passion', 'contempla...
4    {'conveyed_emotions': ['Concern', 'Passion', '...
Name: emotions, dtype: object

# Extracting emotions from the dataset , which is in the form of a list ,dictionary , or nested dictionary 

In [115]:
import pandas as pd
import ast

# Function to process the emotions column
def extract_emotions(emotion_entry):
    emotions = []
    try:
        if isinstance(emotion_entry, dict):  # If it's a dictionary
            for key, value in emotion_entry.items():
                if isinstance(value, list):
                    emotions.extend(value)
        elif isinstance(emotion_entry, list):  # If it's a list
            for item in emotion_entry:
                if isinstance(item, dict):  # Handle nested dictionaries
                    for key, value in item.items():
                        if isinstance(value, list):
                            emotions.extend(value)
                elif isinstance(item, str):  # Handle string elements in list
                    emotions.append(item)
        elif isinstance(emotion_entry, str):  # If it's a string, parse as dictionary if possible
            parsed_entry = ast.literal_eval(emotion_entry)
            if isinstance(parsed_entry, dict):
                for key, value in parsed_entry.items():
                    if isinstance(value, list):
                        emotions.extend(value)
    except Exception as e:
        print(f"Error processing entry {emotion_entry}: {e}")
    return emotions

# Apply the function to the DataFrame
filtered_df['extracted_emotions'] = filtered_df['emotions'].apply(extract_emotions)

# Display the result
print(filtered_df[['username', 'extracted_emotions']])

      username                                 extracted_emotions
0    afrobeezy       [enthusiasm, engagement, curiosity, passion]
1    afrobeezy  [Anger, Determination, Pain, Empowerment, Isol...
2    afrobeezy  [Wonder, Intensity, Compassion, Engagement, Co...
3    afrobeezy               [excitement, passion, contemplation]
4    afrobeezy  [Concern, Passion, Reflectiveness, Urgency, Ho...
..         ...                                                ...
995     shivam                 [encouraging, supportive, helpful]
996     shivam  [deep contemplation, curiosity, nostalgia, awe...
997     shivam  [Focused determination, Inspiring, Calmness, P...
998     shivam  [anticipation, concern, excitement, fear, exhi...
999     shivam                 [suspense, engagement, excitement]

[999 rows x 2 columns]


# Further preprocessing

In [116]:
filtered_df['emotions_conveyed'] = filtered_df['extracted_emotions']

In [117]:
filtered_df[filtered_df['category_name'] == 'Flic']['emotions_conveyed']

filtered_df.to_csv('filtered_df.csv',index=False)

# Content-based recommendation system coupled with Optuna

In [125]:
filtered_df

,title,comment_count,upvote_count,view_count,exit_count,rating_count,average_rating,share_count,username,upvoted,...,description,emotions,isVerified,has_wallet,post_count,following_count,follower_count,extracted_emotions,emotions_conveyed,combined_text
0,Recipe for a flow state,-0.209867,0.728560,0.083248,-0.365784,0.701225,1.801194,7.875664,afrobeezy,3.854496,...,The video unfolds as a dynamic exploration of ...,"{'conveyed': ['enthusiasm', 'engagement', 'cur...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[enthusiasm, engagement, curiosity, passion]","[enthusiasm, engagement, curiosity, passion]",Recipe for a flow state The video unfolds as a...
1,Why fit in..?,-0.209867,0.584881,-0.158744,-0.365784,-0.439428,-0.726536,-0.178374,afrobeezy,-0.259437,...,The video is structured as an emotional and in...,"{'expressed_emotions': ['Anger', 'Determinatio...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[Anger, Determination, Pain, Empowerment, Isol...","[Anger, Determination, Pain, Empowerment, Isol...",Why fit in..? The video is structured as an em...
2,Transcending Singularity,-0.209867,-0.468768,0.405904,-0.365784,6.893341,-0.164818,-0.178374,afrobeezy,-0.259437,...,The video presents a compelling visual narrati...,"{'emotional_content': ['Wonder', 'Intensity', ...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[Wonder, Intensity, Compassion, Engagement, Co...","[Wonder, Intensity, Compassion, Engagement, Co...",Transcending Singularity The video presents a ...
3,Peak Performance?,-0.209867,1.973782,0.943665,-0.209389,0.538275,1.239476,-0.178374,afrobeezy,-0.259437,...,The video presents a seamless narrative explor...,"{'moods': ['excitement', 'passion', 'contempla...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[excitement, passion, contemplation]","[excitement, passion, contemplation]",Peak Performance? The video presents a seamles...
4,Our Existential Situation,-0.209867,1.638530,0.325240,-0.365784,1.027126,1.590550,-0.178374,afrobeezy,-0.259437,...,"The video opens with a male speaker, who has a...","{'conveyed_emotions': ['Concern', 'Passion', '...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[Concern, Passion, Reflectiveness, Urgency, Ho...","[Concern, Passion, Reflectiveness, Urgency, Ho...",Our Existential Situation The video opens with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Kalsubai is the highest peak in Maharashtra, s...",-0.209867,-0.325088,-0.158744,0.911447,-0.113527,1.450120,-0.178374,shivam,-0.259437,...,I'm unable to analyze or interpret video frame...,"{'conveyed_emotions': ['encouraging', 'support...",0.0,1.0,-2.962312,-2.492995,-3.492918,"[encouraging, supportive, helpful]","[encouraging, supportive, helpful]","Kalsubai is the highest peak in Maharashtra, s..."
996,"Tap into the magic of winter. .Sometimes, all ...",-0.209867,-1.091378,0.755449,1.015710,-0.113527,0.888403,-0.178374,shivam,-0.259437,...,"As the video unfolds, it presents a cohesive n...","{'range_of_emotions': ['deep contemplation', '...",0.0,1.0,-2.962312,-2.492995,-3.492918,"[deep contemplation, curiosity, nostalgia, awe...","[deep contemplation, curiosity, nostalgia, awe...","Tap into the magic of winter. .Sometimes, all ..."
997,anzeige -- ️From idea to masterpiece ️with the,-0.209867,-0.277195,0.809225,1.928018,-0.113527,0.396899,-0.178374,shivam,-0.259437,...,"In the video, a cohesive narrative unfolds thr...",{'emotional_expressions': ['Focused determinat...,0.0,1.0,-2.962312,-2.492995,-3.492918,"[Focused determination, Inspiring, Calmness, P...","[Focused determination, Inspiring, Calmness, P...",anzeige -- ️From idea to masterpiece ️with the...
998,"In the heart of darkness, magic is our only sh...",-0.209867,-1.187165,0.621009,0.728985,-0.113527,0.116041,-0.178374,shivam,-0.259437,...,The video unfolds as a cinematic narrative set...,"{'expressed': ['anticipation', 'concern', 'exc...",0.0,1.0,-2.962312,-2.492995,-3.492918,"[anticipation, concern, excitement, fear, exhi...","[anticipation, concern, excite

In [127]:
filtered_df['description']

,title,comment_count,upvote_count,view_count,exit_count,rating_count,average_rating,share_count,username,upvoted,...,description,emotions,isVerified,has_wallet,post_count,following_count,follower_count,extracted_emotions,emotions_conveyed,combined_text
0,Recipe for a flow state,-0.209867,0.728560,0.083248,-0.365784,0.701225,1.801194,7.875664,afrobeezy,3.854496,...,The video unfolds as a dynamic exploration of ...,"{'conveyed': ['enthusiasm', 'engagement', 'cur...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[enthusiasm, engagement, curiosity, passion]","[enthusiasm, engagement, curiosity, passion]",Recipe for a flow state The video unfolds as a...
1,Why fit in..?,-0.209867,0.584881,-0.158744,-0.365784,-0.439428,-0.726536,-0.178374,afrobeezy,-0.259437,...,The video is structured as an emotional and in...,"{'expressed_emotions': ['Anger', 'Determinatio...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[Anger, Determination, Pain, Empowerment, Isol...","[Anger, Determination, Pain, Empowerment, Isol...",Why fit in..? The video is structured as an em...
2,Transcending Singularity,-0.209867,-0.468768,0.405904,-0.365784,6.893341,-0.164818,-0.178374,afrobeezy,-0.259437,...,The video presents a compelling visual narrati...,"{'emotional_content': ['Wonder', 'Intensity', ...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[Wonder, Intensity, Compassion, Engagement, Co...","[Wonder, Intensity, Compassion, Engagement, Co...",Transcending Singularity The video presents a ...
3,Peak Performance?,-0.209867,1.973782,0.943665,-0.209389,0.538275,1.239476,-0.178374,afrobeezy,-0.259437,...,The video presents a seamless narrative explor...,"{'moods': ['excitement', 'passion', 'contempla...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[excitement, passion, contemplation]","[excitement, passion, contemplation]",Peak Performance? The video presents a seamles...
4,Our Existential Situation,-0.209867,1.638530,0.325240,-0.365784,1.027126,1.590550,-0.178374,afrobeezy,-0.259437,...,"The video opens with a male speaker, who has a...","{'conveyed_emotions': ['Concern', 'Passion', '...",0.0,1.0,-2.826908,-2.492995,-2.314324,"[Concern, Passion, Reflectiveness, Urgency, Ho...","[Concern, Passion, Reflectiveness, Urgency, Ho...",Our Existential Situation The video opens with...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Kalsubai is the highest peak in Maharashtra, s...",-0.209867,-0.325088,-0.158744,0.911447,-0.113527,1.450120,-0.178374,shivam,-0.259437,...,I'm unable to analyze or interpret video frame...,"{'conveyed_emotions': ['encouraging', 'support...",0.0,1.0,-2.962312,-2.492995,-3.492918,"[encouraging, supportive, helpful]","[encouraging, supportive, helpful]","Kalsubai is the highest peak in Maharashtra, s..."
996,"Tap into the magic of winter. .Sometimes, all ...",-0.209867,-1.091378,0.755449,1.015710,-0.113527,0.888403,-0.178374,shivam,-0.259437,...,"As the video unfolds, it presents a cohesive n...","{'range_of_emotions': ['deep contemplation', '...",0.0,1.0,-2.962312,-2.492995,-3.492918,"[deep contemplation, curiosity, nostalgia, awe...","[deep contemplation, curiosity, nostalgia, awe...","Tap into the magic of winter. .Sometimes, all ..."
997,anzeige -- ️From idea to masterpiece ️with the,-0.209867,-0.277195,0.809225,1.928018,-0.113527,0.396899,-0.178374,shivam,-0.259437,...,"In the video, a cohesive narrative unfolds thr...",{'emotional_expressions': ['Focused determinat...,0.0,1.0,-2.962312,-2.492995,-3.492918,"[Focused determination, Inspiring, Calmness, P...","[Focused determination, Inspiring, Calmness, P...",anzeige -- ️From idea to masterpiece ️with the...
998,"In the heart of darkness, magic is our only sh...",-0.209867,-1.187165,0.621009,0.728985,-0.113527,0.116041,-0.178374,shivam,-0.259437,...,The video unfolds as a cinematic narrative set...,"{'expressed': ['anticipation', 'concern', 'exc...",0.0,1.0,-2.962312,-2.492995,-3.492918,"[anticipation, concern, excitement, fear, exhi...","[anticipation, concern, excite

In [128]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import numpy as np
import optuna
from IPython.display import display,Markdown

# Assuming 'filtered_df' is the DataFrame that holds your data

# Combine title, description, and category_name for text-based features
filtered_df['combined_text'] = (
    filtered_df['title'] + ' ' +
    filtered_df['description'] + ' ' +
    filtered_df['category_name'] + ' ' +
    filtered_df['emotions_conveyed'].astype(str)
)

# Vectorize the combined text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_df['combined_text'])

# Normalize numerical features
numerical_columns = ['upvote_count', 'view_count', 'comment_count', 'rating_count', 'average_rating', 
                     'following', 'exit_count', 'post_count', 'share_count', 
                     'following_count', 'follower_count', 'upvoted', 'bookmarked']
scaler = StandardScaler()
filtered_df[numerical_columns] = scaler.fit_transform(filtered_df[numerical_columns])

def build_user_profile(user, df, weights):
    df = df.reset_index(drop=True)
    user_posts = df[df['username'] == user]
    weighted_vectors = []

    for index, row in user_posts.iterrows():
        interaction_weight = (
            (row['upvote_count'] * weights[0] + row['comment_count'] * weights[1] + row['view_count'] * weights[2])
            + (row['rating_count'] * row['average_rating'] * weights[3])
            + (row['post_count'] * weights[4] + row['share_count'] * weights[5])
        )
        post_vector = tfidf_matrix[index].toarray()
        weighted_vectors.append(post_vector.flatten() * interaction_weight)

    # Aggregate the weighted post vectors to build the user profile
    user_profile = np.mean(weighted_vectors, axis=0) if weighted_vectors else np.zeros(tfidf_matrix.shape[1])
    return user_profile

def get_recommendations_for_user(user, category, mood, top_n, weights):
    if user not in filtered_df['username'].unique():
        # Handle new user case
        category_filtered = filtered_df[filtered_df['category_name'].str.contains(category, case=False, na=False)]
        mood_filtered = category_filtered[category_filtered['emotions_conveyed'].str.contains(mood, case=False, na=False)]
        recommendations_df = mood_filtered if not mood_filtered.empty else category_filtered
        recommendations = recommendations_df[['title', 'description']].head(top_n).values.tolist()
        return recommendations
    else:
        # Handle existing user case
        user_profile = build_user_profile(user, filtered_df, weights)
        cosine_similarities = cosine_similarity(user_profile.reshape(1, -1), tfidf_matrix).flatten()
        recommended_indices = cosine_similarities.argsort()[-top_n:][::-1]

        recommendations = []
        for idx in recommended_indices:
            if filtered_df.loc[idx, 'username'] != user:
                recommendations.append(
                    [filtered_df.loc[idx, 'title'], filtered_df.loc[idx, 'description']]
                )

        return recommendations


def calculate_mae(predictions, ground_truth):
    prediction_set = set(predictions)
    ground_truth_set = set(ground_truth)
    errors = len(ground_truth_set) - len(prediction_set.intersection(ground_truth_set))
    mae = abs(errors) / len(ground_truth_set) if ground_truth_set else 0
    return mae

def calculate_rmse(predictions, ground_truth):
    prediction_set = set(predictions)
    ground_truth_set = set(ground_truth)
    errors = len(ground_truth_set) - len(prediction_set.intersection(ground_truth_set))
    rmse = np.sqrt(errors ** 2 / len(ground_truth_set)) if ground_truth_set else 0
    return rmse

# Define the objective function for Optuna
def optuna_objective(trial):
    # Suggest weights for optimization
    weights = [
        trial.suggest_float(f"weight_{i}", 0.0, 10.0) for i in range(6)
    ]

    # Get recommendations for the current user
    recommendations = get_recommendations_for_user(username, category, mood, len(yhat), weights)

    # Calculate MAE and RMSE
    mae = calculate_mae(recommendations, yhat)
    rmse = calculate_rmse(recommendations, yhat)

    # Return the combined objective (to minimize)
    return mae + rmse

# Inputs
username = input("Enter the username: ")
category = input("Enter the category: ")

mood = input("Enter the mood: ")

# Define yhat based on the user’s posts (ground truth titles)
yhat = filtered_df[filtered_df['username'] == username]['title'].tolist() if username in filtered_df['username'].unique() else []

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(optuna_objective, n_trials=50)

# Get the best weights
optimized_weights = [study.best_params[f"weight_{i}"] for i in range(6)]

# Get recommendations based on optimized weights
recommendations = get_recommendations_for_user(username, category, mood, len(yhat) or 5, optimized_weights)

# Calculate MAE and RMSE
mae = calculate_mae([rec[0] for rec in recommendations], yhat)  # Extract titles for MAE
rmse = calculate_rmse([rec[0] for rec in recommendations], yhat)  # Extract titles for RMSE

# Output the results
print(f"Optimized Weights: {optimized_weights}")
print(f"Mean Absolute Error for recommendations: {mae}")
print(f"Root Mean Squared Error for recommendations: {rmse}")

# Print the recommendations with titles and descriptions using Markdown
print("Recommendations:")
for rec in recommendations:
    # Display title as main heading (H1) and description in markdown
    display(Markdown(f"# {rec[0]}"))  # Title as big heading
    display(Markdown(f"**Description:**\n{rec[1]}"))  # Description in markdown format
    print("------")

print(f"Ground Truth (yhat): {yhat}")

Enter the username:  john
Enter the category:  InstaRama
Enter the mood:  happy


[I 2024-12-12 15:57:06,265] A new study created in memory with name: no-name-243f1cc8-a013-47b4-9568-123f526a51d7
[I 2024-12-12 15:57:06,274] Trial 0 finished with value: 0.0 and parameters: {'weight_0': 8.48553939830544, 'weight_1': 7.344317818281225, 'weight_2': 6.055384785417994, 'weight_3': 6.818099426820906, 'weight_4': 0.1248530347602117, 'weight_5': 1.5297122157753529}. Best is trial 0 with value: 0.0.
[I 2024-12-12 15:57:06,281] Trial 1 finished with value: 0.0 and parameters: {'weight_0': 7.547089142287867, 'weight_1': 2.1491178626430396, 'weight_2': 8.304590419829307, 'weight_3': 0.8602576892627645, 'weight_4': 5.385498816256415, 'weight_5': 5.167286996452635}. Best is trial 0 with value: 0.0.
[I 2024-12-12 15:57:06,289] Trial 2 finished with value: 0.0 and parameters: {'weight_0': 5.120642690079307, 'weight_1': 7.18797346896337, 'weight_2': 8.318165917964947, 'weight_3': 8.40393300487998, 'weight_4': 2.3095497905582496, 'weight_5': 5.09719780290391}. Best is trial 0 with val

Optimized Weights: [8.48553939830544, 7.344317818281225, 6.055384785417994, 6.818099426820906, 0.1248530347602117, 1.5297122157753529]
Mean Absolute Error for recommendations: 0
Root Mean Squared Error for recommendations: 0
Recommendations:


# Jai Shree Ram

**Description:**
Error generating description: Request timed out.

...

**Description:**
Error generating description: Request timed out.

------


# Ram Siya Ram Siya Ram Jai Jai Ram

**Description:**
I'm unable to analyze the video content or the specific audio transcription without the actual multimedia files or comprehensive details about their content. If you have specific elements of the video you can describe, I can help create a storyboard or analyze the narrative based on that information. Please provide more context or details for a thorough analysis!

------
Ground Truth (yhat): []


In [129]:
filtered_df['category_name'].value_counts()

category_name
Vible              528
E/ACC              207
InstaRama          169
SolTok              48
Pumptok             19
Flic                19
OvaDrive             5
Gratitube            2
The Igloo            1
Startup College      1
Name: count, dtype: int64